In [1]:
import os
import time

import pandas as pd
import FinanceDataReader as fdr
import datetime
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import numpy as np
import argparse
from copy import deepcopy 
from sklearn.metrics import mean_absolute_error

import seaborn as sns 
import matplotlib.pyplot as plt

In [2]:
%matplotlib inline

plt.rcParams["figure.figsize"] = (14,8)
plt.rcParams['font.size'] = 16
plt.rcParams['lines.linewidth'] = 2
plt.rcParams["axes.grid"] = True
plt.rcParams['axes.axisbelow'] = True

In [716]:
def get_data(sym):
    file_path = "./stockData/"
    csv_path = os.path.join(file_path, f"{sym}.csv")
    df = pd.read_csv(csv_path, parse_dates=True, index_col = ['Date'])
    df.drop('Symbol', axis=1, inplace=True)
    return df

In [5]:
sp500 = fdr.StockListing('S&P500')
sp500_list = list(sp500['Symbol'])

In [6]:
price_dict = {}

for i, data in sp500.iterrows():
    sym, name = data['Symbol'], data['Name']
    df =  get_data(sym)
    price_dict[sym] = df['Close']

In [848]:
df_price = pd.DataFrame()

for i, data in sp500.iterrows():
    sym, name = data['Symbol'], data['Name']
    df =  get_data(sym)
    df_price[sym] = df['Close']

In [854]:
df_price

,MMM,ABT,ABBV,ABMD,ACN,ATVI,ADBE,AMD,AAP,AES,...,WYNN,XEL,XRX,XLNX,XYL,YUM,ZBRA,ZBH,ZION,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2000-01-03,47.19,15.66,NaN,18.25,NaN,1.37,16.39,15.50,NaN,36.25,...,NaN,19.00,24.19,46.75,NaN,9.33,25.03,NaN,55.50,NaN
2000-01-04,45.31,15.22,NaN,17.81,NaN,1.33,15.02,14.62,NaN,34.81,...,NaN,19.44,23.06,45.69,NaN,9.14,24.67,NaN,52.81,NaN
2000-01-05,46.62,15.19,NaN,18.00,NaN,1.33,15.31,15.00,NaN,35.19,...,NaN,20.19,24.25,44.69,NaN,9.19,25.14,NaN,52.75,NaN
2000-01-06,50.38,15.72,NaN,18.03,NaN,1.31,15.44,16.00,NaN,35.47,...,NaN,20.00,23.75,40.81,NaN,9.11,23.78,NaN,53.50,NaN
2000-01-07,51.38,15.89,NaN,17.94,NaN,1.34,16.19,16.25,NaN,36.16,...,NaN,20.00,24.31,45.44,NaN,8.91,23.51,NaN,53.62,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-18,176.42,108.97,104.45,293.39,266.25,90.37,502.95,95.92,159.99,22.54,...,115.93,66.10,22.86,149.19,100.05,109.78,373.00,151.85,41.08,163.63
2020-12-21,175.66,108.09,103.71,295.81,261.42,88.81,498.39,93.23,160.49,22.62,...,112.54,65.39,22.38,145.28,100.72,108.63,375.91,148.60,42.43,160.84
2020-12-22,174.78,108.28,102.80,302.33,259.47,89.42,504.17,93.16,159.40,22.82,...,112.61,64.83,22.38,144.16,99.40,107.40,379.96,150.83,42.04,161.83


In [593]:
collect = []

In [607]:
best_list = []

for _ in range(1000):
    corr_price = df_price_corr.copy()

    cor_dict = []

    rnd = []
    
    first = []

    for num in range(10):
        temp = np.random.randint(1,505 - (num * 51))
        rnd.append(temp)

    for r in rnd:
        i = corr_price.columns.values[r]
        first.append(i)

        temp = corr_price[i]
        temp_sort = temp.sort_values(ascending=False)
        cor_dict.append(temp_sort[1:31])

        idx = temp_sort[0:31].index.values
        corr_price.drop(idx, inplace=True)
        corr_price.drop(idx, axis=1, inplace=True)

    remain = list(corr_price.columns.values)
    
    for id, col_name in enumerate(reversed(first)):
        id = len(first)-id-1
        result = pd.DataFrame()
        result = df_price[col_name]

        for i in remain:
            result = pd.concat([result, df_price[i]], axis = 1)

        result_corr = result.corr()
        st = result_corr.columns.values[0]


        temp = result_corr[st]
        temp_sort = temp.sort_values(ascending=False)
        cor_dict[id]=cor_dict[id].append(temp_sort[1:21])

        idx = list(temp_sort[1:21].index.values) 

        for rmv in idx:
            remain.remove(rmv)
        
    best_list.append(cor_dict)

In [608]:
scores = []

for co_list in best_list:
    t = []
    for cor in co_list:
        corr_mean = cor.mean()
        t.append(corr_mean)
    mean_score = np.mean(t)
    scores.append((mean_score))
    
print(np.max(scores))
best_idx = np.argmax(scores)
print(best_idx)

0.801708625930412
4


In [609]:
a = best_list[best_idx]
collect.append(a)

for c in a:
    b = c.mean()
    print(b)

0.5997848092229028
0.7016073592040967
0.8197321795846427
0.8544774586418632
0.924168776526368
0.5722124245447572
0.8559701234239175
0.9024088484101985
0.9031671423631841
0.8835571373821892


In [606]:
for c in collect[3]:
    b = c.mean()
    print(b)

0.7929825234667038
0.7638970641985822
0.8584003186817633
0.9300870972479837
0.8039574033903052
0.9344076463156373
0.8750198200188432
0.6226147343479097
0.7431588856809388
0.9518549368822753


In [627]:
my_data_set = []

for c in collect[3]:
    temp = list(c.index.values)
    temp.append(c.name)
    my_data_set.append(temp)

In [852]:
import hashlib
import json
from os import listdir
from os.path import isfile, join


def save_exp_result(result):    
    filepath = './results'
    if not os.path.isdir(filepath):
        os.mkdir(filepath)

    filename = './stockData/sp500_list.json'
    with open(filename, 'w') as f:
        json.dump(result, f)

In [632]:
save_exp_result(my_data_set)

In [853]:
save_exp_result(sp500_list)

In [637]:
def load_data_set(filename):
    dir_path = './data_set'
    
    with open(join(dir_path, filename), 'r') as f:
        results = json.load(f)
        
    return results 

In [639]:
result = load_data_set('my_data_set.json')